**DAT405 Introduction to Data Science and AI, 2021, Study Period 2** <br/>
**Assignment 5: Reinforcement learning and Classification** <br/>
**Due Date: Dec 7, 23:59** <br/>

**Carl Kindberg: At least 25 Hours**





---
This assignment is about sequential decision making under uncertainty (Reinforcement learning). In a sequential decision process, in each state the decision maker, or agent, chooses among a set of actions, and the system (environment) jumps to a new state based on both the current state and the chosen action. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal policy) that maximizes the rewards. 

We will use Markov decision processes (MDPs) to model the environment, and the assignment is divided into two parts: 
-	First, we focus on a decision process with no uncertainty, meaning that we can compute the optimal action in each state. We will use an MDP to model the environment, and then introduce one algorithm (out of many) for finding the optimal policy. 
- Next, we will use Q-learning to make decisions under uncertainty. Q-learning is a reinforcement learning method that can be used to explore and learn about an unknown environment. The objective is again to determine an optimal policy for the now unknown MDP.

We have attached an example notebook illustrating the use of the OpenAI gym library used in questions 3 to 5 (q_learning_frozen_lake.ipynb).

## What to Submit

**The entire assignment should be submitted through the notebook. No separate file will be accepted.** 
You have the following options to submit the assignment:
-	Submit a link to a completed and fully executed Google Colab notebook (please make sure it is executable and editable for anybody with the link).
-	Submit a completed and fully executed Jupyter notebook (.ipynb-file) from Colab or in Jupyter.


**The notebooks should be executed and all code output should be visible and readable.** In Google Colab, first go to the Runtime menu and select Factory Reset-Runtime and then go to the Runtime again and select Run all.


*In the notebook:*
*	State your names and how many hours each person spent on the assignment.
*	The solutions and answers to the theoretical and practical problems, including LaTeX math-mode equations, plots and tables etc.
*	All plots/results should be visible such that the notebook does not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.<br/>

*Before submitting:*
*   Make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.

**Self-check**<br/>
Is all the required information included? Have you answered all questions to the best of your ability? Anything else you can easily check? (details, terminology, arguments, clearly stated answers etc.?) Does your notebook run and can reproduce the results, plots and tables?

**Grading**<br/>
Grading will be based on a qualitative assessment of each assignment. It is important to:
*	Present clear arguments
*	Present the results in a pedagogical way
*	Show understanding of the topics (e.g, write a pseudocode) 
*	Give correct solutions
*	Make sure that the code is well commented 

**Again, as mentioned in general guidelines, all code should be written here. And this same ipython notebook file (Assignment5_RL.ipynb) should be submitted with answers and code written in it. No separate file will be accepted.** 


# Primer

## Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.


## Overview
* To make things concrete, we will first focus on decision making under **no** uncertainity, i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.


* Next, we will work through one type of reinforcement learning algorithm called Q-learning. Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

## Markov Decision Process

Markov Decision Process (MDP) provides a mathematical framework for modeling sequential decision making under uncertainty. A MDP consists of five parts: the specific decision times, the state space of the environment/system, the available actions for the decision maker, the rewards, and the transition probabilities between the states.

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ for the current state and action, and the resulting next state
* Transition probabilities $p(s'|s,a)$ that taking action $a$ in state $s$ will lead to state $s'$

At a given decision epoch $t$ and system state $s_t$, the decions maker, or *agent*, chooses an action $a_t$, the system jumps to a new state $s_{t+1}$ according to the transition probability $p(s_{t+1}|s_t,a_t)$, and the agent receives a reward $r_t(s_t,a_t,s_{t+1})$. This process is then repeated for a finite or infinite number of times.

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if you think all future rewards should count equally, you would use $\gamma = 1$, while if you only care less about future rewards you would use $\gamma < 1$. The expected total *discounted* reward becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^{\pi^*}(s)$ for all $s\in S$. That is

$$V^{\pi^*}(s) \geq V^\pi(s), s\in S$$

The problem of finding the optimal policy is a _dynamic programming problem_. It turns out that a solution to the optimal policy problem in this context is the *Bellman equation*. The Bellman equation is given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

Thus, it can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. Your states would be the amount of items you have in stock. Your actions would be the amount to order. The discrete time would be the days of the month. The reward would be the profit.  

A major drawback of MDPs is called the "Curse of Dimensionality". MDPs unfortunately do not scale very well with increasing sets of states or actions.   


## Question 1

In this first question we work with the deterministic MDP, no code is necessary in this part.

Setup:

* The state space is a grid with 3 columns and 4 rows, where each state is identified by its position. That is, each state has the form $s = (x,y)$ for $x\in\{0,1,2\}$ and $y \in\{0,1,2,3\}$.
* The agent starts in state **A** $=(0,0)$
* The actions possible are to move **N** (north), **S** (south), **E** (east), and **W** west.
* Note, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **B** = $(2,3)$, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$

The reward of a state $r(s=(x, y))$ is given by the values on the grid:
    
| | | |
|----------|----------|---------|
|-1 |1|**B**|
|0|-1|1|  
|-1 |0|-1|  
|**A**|-1|1|




**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.


A = (0,0) --> (1,0) --> (2,0) --> (2,1) --> (2,2) --> (2,3) = B, is the shortest optimal path i.e EENNN. Reward = -1+1-1+1 = 0

A = (0,0) --> (1,0) --> (2,0) --> (2,1) --> (2,2) --> (1,2) --> (1,3) --> (2,3) = B i.e EENNWNE Reward = -1+1-1+1-1+1 =0

**Since we can find two paths that gives the same results the MDP isn't unique.**

**1b)** What is the optimal policy (i.e. the optimal action in each state)? Is it unique?



    
|Current state | Action (N,S,E,W) |Reward (-1, 0, 1, **End**) |
|----------|----------|---------|
|**A**=(0,0)| N,E | -1 |
|(0,1)| N,E | 0 |
|(0,2)| N,E,S| -1 |
|(0,3)| E | 1 |
|(1,0)| E | 1 |
|(1,1)| N,S,E,W | -1 |
|(1,2)| E | 1 |
|(1,3)| W | **End** |
|(2,0)| N,W | -1 |
|(2,1)| N,S | 1 |
|(2,2)| N | **End** |
|**B**=(2,3)| **End** | **End** |



| | | |
|----------|----------|---------|
|(0,3)|(1,3)|(2,3)|
|(0,2)|(1,2)|(2,2)|  
|(0,1)|(1,1)|(2,1)|  
|(0,0)|(1,0)|(2,0)|

**1c)** What is expected total reward for the policy in 1b)?

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V$ in the right-hand side (RHS) of the Bellman equation should result in the same $V$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V^*$ which satisfies the Bellman equation. Hence repeated application of the RHS of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what action $a$ achieves the maximum on the RHS of the Bellman equation applied to $V^*$. The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm.

The value iteration algorithm practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```






**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards $r(s=(x,y))$ given in each state:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0| 

Assume now that given a state $s$ and action $a$, there is a probability of 0.8 that that action will be performed and a probability of 0.2 that no action is taken. For instance, if we take action **E** in state $s=(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state, that is, we stay on the grid), and remain still 20 percent of the time. We will use a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

Thus, writing the initial value function $V^0(s)$ in each state $s=(x,y)$ in the grid gives:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|0|0|  
|0|0|0| 


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0(s')$ was zero for all $s'$. 

The updated values $V^1(s)$ for each state become:

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Starting with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us the updated value function $V^2(s)$:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Implement the value iteration algorithm just described here in python, and show the converging optimal value function and the optimal policy for the above 3x3 grid. Hint: use the pseudo-code above as a starting point, but be sure to explain what every line does.


In [603]:
# Used packages
import itertools
import numpy as np
import copy

In [604]:
# Function to decide wheter a move is possible or not, S = (X_i,Y_j)
def pos_move(i, j, num_rows, num_cols):
    pos_moves = []

    # SOUTH    
    if j-1 >= 0:
        pos_moves.append([i,j-1])
    
    # EAST
    if i+1 < num_cols:
        pos_moves.append([i+1,j])
    
    # NORTH
    if j+1 < num_rows:
        pos_moves.append([i,j+1])
      
    # WEST
    if  i-1 >= 0:
        pos_moves.append([i-1,j])
            
    return pos_moves

In [605]:
# Calculating V
def V_func(i, j, V_prev, reward, gamma, p):

    # Initialize V =0
    V = 0  
    
    
    for i_,j_, in pos_move(i, j, num_rows = 3, num_cols = 3):
        
        # initialize v_temp
        v_temp = 0
        
        #v_temp += p*(reward[j_][i_]+gamma*V_prev[j_][i_]) + (1-p)*(reward[j][i] + gamma*V_prev[j][i])
        v_temp += p*(reward[i_][j_]+gamma*V_prev[i_][j_]) + (1-p)*(reward[i][j] + gamma*V_prev[i][j])
#         print("v_temp",v_temp)
       
        if v_temp >= V:
            V = v_temp

    return V

In [610]:
# Finally let's define the value iteration function! 
def VI(V0, reward, eps, gamma, num_rows, num_cols):
   
    # Create zero matrix
    V = np.zeros(shape = (num_rows, num_cols))
    # Set v_prev to the V_0
    V_prev = V0
    
    # iter count
    iter = 0
 
    # infinite loop
    for _ in itertools.count(): 
        iter = iter +1
        
        # Loop through num_rows and cols, Stop only if abs(V - V_previous) < eps otherwise continue!
        for i in range(num_rows):
#             print(i)
            
            for j in range(num_cols):
#                 print(j)
                
                V[i][j] = V_func(i,j, V_prev, reward, gamma, p)
#                 print(V)
                
            if (np.abs(V - V_prev) < eps).all():
                return (V), iter
        
        V_prev = copy.deepcopy(V)

In [611]:
# Okay, now we can finally run the code to get  nice result except that
# I didn't manage to solve the Optimal policy..
# Parameters used:
V0 = [[0,0,0], [0,0,0], [0,0,0]]
reward = [[0,0,0],[0,10,0], [0,0,0]]
eps = 0.000000000000001
gamma = 0.9
num_rows = len(reward)
num_cols = len(reward[0])
p = 0.8

final_iteration, num_iterations = VI(V0, reward, eps, gamma, num_rows, num_cols)

print("Number of iterations to convergence:")
print(num_iterations)

print("Final iteration:")
print(final_iteration)


Number of iterations to convergence:
334
Final iteration:
[[45.61292366 51.94805195 45.61292366]
 [51.94805195 48.05194805 51.94805195]
 [45.61292366 51.94805195 45.61292366]]


As we can see the result is symmetric and thus seems reasonable as we started with a reward matrix that was symmetric.
It took 334 iterations before the Value iteration converged.

**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

**Answer 2b)**
The result in 2a) doesn't depend on the value of $V_0$ as the iteration goes to infinty if $\gamma<1$. Why? well since we multiply with a discount factor that is less than one the inital value becomes "redundant" (maybe not the right word choice) as iteration goes to infinty.

## Reinforcement Learning (RL)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(a,s,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(a,s,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(a,s
]\,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

## Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there is sufficient exploration. While a constant $\alpha$ generally does not guarantee us to reach true convergence, we keep it constant at $\alpha=0.1$ for this assignment.

## OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular OpenAI Gym library. It just implements [different types of simulators](https://gym.openai.com/) including ATARI games. Although here we will only focus on simple ones, such as the [Chain enviroment](https://gym.openai.com/envs/NChain-v0/) illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.



## Question 3
**3a)** You are to first familiarize with the framework using it's [documentation](http://gym.openai.com/docs/), and then implement the Q-learning algorithm for the Chain enviroment (called 'NChain-v0') using default parameters. Finally print the $Q^*$ table at convergence. Convergence is **not** a constant value, rather a stable plateau with some noise. Take $\gamma=0.95$. You can refer to the Q-learning (frozen lake) Jupyter notebook shown in class, uploaded on Canvas. Hint: start with a small learning rate.

In [380]:
# Used packages'
import gym
import gym_toytext
import numpy as np
import random
import math
import time

In [381]:
# Importing the NChain-v0 enviorment, Documentation can be found above
env = gym.make("NChain-v0")

In [382]:
num_episodes = 100 #15000 20000 #60000
gamma = 0.95 #0.99
learning_rate = 0.7 #0.95 #0.85
epsilon = 0.5 #1 #0.15 #0.1

# initialize the Q table
Q = np.zeros([5, 2])
print(Q)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [96]:


def func(num_episodes, gamma, learning_rate, epsilon, Q, env, runs):   
    Total_start = time.time()    
    
    
    Q_res = []
    for i in range(0,runs):
        start = time.time()
        print("Run:", i+1)
        
        # Everything in this loop is from q_learning_frozen_lake
        for _ in range(num_episodes):
            state = env.reset()
            done = False
            while done == False:
                # First we select an action:
                if random.uniform(0, 1) < epsilon: # Flip a skewed coin
                    action = env.action_space.sample() # Explore action space
                else:
                    action = np.argmax(Q[state,:]) # Exploit learned values
                # Then we perform the action and receive the feedback from the environment
                new_state, reward, done, info = env.step(action)
                # Finally we learn from the experience by updating the Q-value of the selected action
                update = reward + (gamma*np.max(Q[new_state,:])) - Q[state, action]
                Q[state,action] += learning_rate*update 
                state = new_state
        
        Q_res.append(Q)
        print("Elapsed time:", (time.time() - start),"sec") 
    print("Average Q:") 
    print(sum(Q_res)/runs)
    print("Elapsed time:", (time.time() - start),"sec") 
    
    
    print("Total elapsed time:", (time.time() - Total_start),"sec") 

In [583]:
func(num_episodes = 1000, 
     gamma = 0.95, 
     learning_rate = 0.1, 
     epsilon = 0.5, 
     Q = np.zeros(shape = (5,2)), 
     env = gym.make("NChain-v0"),
     runs = 5)

Run: 1
Elapsed time: 12.6798837184906 sec
Run: 2
Elapsed time: 12.68709945678711 sec
Run: 3
Elapsed time: 13.20667839050293 sec
Run: 4
Elapsed time: 12.527976989746094 sec
Run: 5
Elapsed time: 12.841877698898315 sec
Average Q:
[[62.20615165 61.84659439]
 [65.21933741 62.69444237]
 [68.75646754 63.44610952]
 [72.50107524 65.31054311]
 [77.72192582 66.73871398]]
Elapsed time: 12.841877698898315 sec
Total elapsed time: 63.945478200912476 sec


**3b)** Does the result seem reasonable? Why?

Your answer here.

## Question 4

**4a)** Define the MDP corresponding to the Chain environment above and verify that the optimal $Q^*$ value obtained using simple Q-learning is the same as the optimal value function $V^*$ for the corresponding MDP's optimal action. Hint: compare values obtained using value iteration and Q-learning.



In [586]:
# Function to decide wheter a move is possible or not, S = (X_i,Y_j)
def pos_move(i, j, num_rows = 5, num_cols=2):
    pos_moves = []

    # SOUTH    
    if j-1 >= 0:
        pos_moves.append([i,j-1])
    
    # EAST
    if i+1 < num_cols:
        pos_moves.append([i+1,j])
    
    # NORTH
    if j+1 < num_rows:
        pos_moves.append([i,j+1])
      
    # WEST
    if  i-1 >= 0:
        pos_moves.append([i-1,j])
            
    return pos_moves

In [591]:
# Finally let's define the value iteration function! 
def VI(V0, reward, eps, gamma, num_rows=5, num_cols=2):
   
    # Create zero matrix
    V = np.zeros(shape = (5, 2))
    #V = [[0,0], [0,0], [0,0], [0,0], [0,0]]
    #V = [[0,0,0,0,0], [0,0,0,0,0]]
    
    # Set v_prev to the V_0
    V_prev = V0 # maybe not the right name for this variable ... 
    
    # iter count
    iter = 0
 
    # infinite loop
    
    for _ in itertools.count():
        iter = iter +1
        
        # Loop through num_rows and cols, Stop only if abs(V - V_previous) < eps otherwise continue!
        for i in range(0,5):
#             print("i:", i)
                
            for j in range(0,2):
#                 print("j:", j)
            
        
                V[i][j] = V_func(i,j, V_prev, reward, gamma, p)
                
                #print("V:",V)
                #print("V_PREV:", V_prev)

                
            if (np.abs((V) - (V_prev)) < eps).all():
                return (V), iter
        
        V_prev = copy.deepcopy(V)

In [592]:
# Calculating V
def V_func(i, j, V_prev, reward, gamma, p):

    # Initialize V =0
    V = 0  
    
    
    for i_,j_, in pos_move(i, j, num_rows = 2, num_cols = 5):
#         print("i_:", i_)
#         print("i:", i)
#         print("j_:", j_)
#         print("j:", j)
    
        
        # initialize v_temp
        v_temp = 0
        
        #v_temp += p*(reward[j_][i_]+gamma*V_prev[j_][i_]) + (1-p)*(reward[j][i] + gamma*V_prev[j][i])
        v_temp += p*(reward[i_][j_]+gamma*V_prev[i_][j_]) + (1-p)*(reward[i][j] + gamma*V_prev[i][j])
#         print("v_temp",v_temp)
       
        if v_temp >= V:
            V = v_temp

    return V

In [595]:

# Parameters used:
#V0 = [[0,0], [0,0], [0,0], [0,0], [0,0]]
#V0 = [[0,0,0,0,0], [0,0,0,0,0]]
V0 = np.zeros(shape = (5,2))
#reward = [[0,2],[0,2], [0,2], [0,2], [0,10]]
#reward = [[0,0,0,0,0], [2,2,2,2,10]]
reward = np.zeros(shape = (5,2))
reward[0][1] = 2
reward[1][1] = 2
reward[2][1] = 2
reward[3][1] = 2
reward[4][1] = 10

eps = 0.5
gamma = 0.95
num_rows = len(reward)
num_cols = len(reward[0])
p = 0.8

final_iteration, num_iterations = VI(V0, reward, eps, gamma, num_rows=2, num_cols=5)

print("Number of iterations to convergence:")
print(num_iterations)

print("Final iteration:")
print(final_iteration)


Number of iterations to convergence:
50
Final iteration:
[[ 93.45932107  98.11015778]
 [ 97.13036236 102.05468654]
 [101.56085938 106.77666364]
 [106.28283648 111.80929712]
 [111.31546996 108.75197228]]


Okay, so I actually didn't get exactly the same results as I recieved with the Q-learning algorithm. I don't know why but it might be some minor mistakes with the indexing as a I had a hard time getting the algorithm to work.

**Please leave a comment if you find the error in my code :)**

**4b)** What is the importance of exploration in RL? Explain with an example.

Actually didn't have the time as I spent way to many hours on getting the Value iteration to work...

## Question 5

**5a)** Explain how a decision tree works and how it extends to random forests.



Your answer here.

**5b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification.

Your answer here.


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf